<a href="https://colab.research.google.com/github/Kshitij111111/ConversationMgmt_Groq/blob/main/ConversationMgmt_Groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab cell (bash)
!pip install --upgrade openai jsonschema


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 947.6/947.6 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.106.1
    Uninstalling openai-1.106.1:
      Successfully uninstalled openai-1.106.1


In [2]:
# Colab cell (python)
import os
from getpass import getpass

# Preferred (secure) way — paste when prompted (won't echo)
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass("Paste your GROQ API key (it will not show): ")

print("GROQ_API_KEY loaded into environment (not printed).")


Paste your GROQ API key (it will not show): ··········
GROQ_API_KEY loaded into environment (not printed).


In [3]:
# Colab cell (python)
import os, json
from openai import OpenAI

# Create client that talks to Groq using the OpenAI-compatible SDK
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise RuntimeError("Set GROQ_API_KEY in environment first")

client = OpenAI(api_key=GROQ_API_KEY, base_url="https://api.groq.com/openai/v1")

# quick sanity check (do not print your key)
print("Client created. (Using Groq OpenAI-compatible endpoint).")


Client created. (Using Groq OpenAI-compatible endpoint).


# Colab cell (python)
import os, json
from openai import OpenAI

# Create client that talks to Groq using the OpenAI-compatible SDK
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise RuntimeError("Set GROQ_API_KEY in environment first")

client = OpenAI(api_key=GROQ_API_KEY, base_url="https://api.groq.com/openai/v1")

# quick sanity check (do not print your key)
print("Client created. (Using Groq OpenAI-compatible endpoint).")


In [4]:
# Colab cell (python)
import json
from typing import List, Dict

class ConversationManager:
    def __init__(self):
        self.messages: List[Dict] = []  # list of {"role": "user"/"assistant"/"system", "content": str}
        self.run_count = 0

    def add_user(self, text: str):
        self.messages.append({"role": "user", "content": text})

    def add_assistant(self, text: str):
        self.messages.append({"role": "assistant", "content": text})

    def add_system(self, text: str):
        self.messages.insert(0, {"role": "system", "content": text})  # keep at top if needed

    def last_n(self, n: int):
        return self.messages[-n:]

    def truncate_by_chars(self, max_chars: int):
        # Keep messages from the end until combined chars <= max_chars
        out = []
        total = 0
        for m in reversed(self.messages):
            l = len(m['content'])
            if total + l > max_chars:
                break
            out.append(m)
            total += l
        self.messages = list(reversed(out))

    def truncate_by_words(self, max_words: int):
        out = []
        total = 0
        for m in reversed(self.messages):
            w = len(m['content'].split())
            if total + w > max_words:
                break
            out.append(m)
            total += w
        self.messages = list(reversed(out))

    def keep_last_n(self, n: int):
        self.messages = self.last_n(n)

    def save(self, path="conversation_history.json"):
        with open(path, "w", encoding="utf-8") as f:
            json.dump(self.messages, f, ensure_ascii=False, indent=2)

    def load(self, path="conversation_history.json"):
        with open(path, "r", encoding="utf-8") as f:
            self.messages = json.load(f)


In [21]:
def summarize_conversation(client, conv_manager: ConversationManager,
                           model="llama-3.1-8b-instant", max_tokens=200, temperature=0.1):
    """
    Summarizes the full conversation so far into a concise version.
    """
    convo_text = "\n\n".join([f"{m['role']}: {m['content']}" for m in conv_manager.messages])
    system_prompt = (
        "You are a concise summarizer. Produce a short summary (2-4 sentences) of the conversation, "
        "focusing on user intent and important entities."
    )
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": convo_text}
    ]

    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )
    summary = resp.choices[0].message.content

    return summary.strip()



In [22]:
def run_with_periodic_summarization(client, conv_manager: ConversationManager, user_input: str,
                                    k=3, summarizer_model="llama-3.1-8b-instant"):
    """
    Adds user input, generates assistant response, and applies summarization
    every k-th run.
    """
    conv_manager.add_user(user_input)

    # Assistant response
    resp = client.chat.completions.create(
        model=summarizer_model,
        messages=conv_manager.messages,
        max_tokens=300,
        temperature=0.2
    )
    assistant_text = resp.choices[0].message.content

    conv_manager.add_assistant(assistant_text)
    conv_manager.run_count += 1

    # Summarize every k-th run
    if conv_manager.run_count % k == 0:
        summary = summarize_conversation(client, conv_manager,
                                         model=summarizer_model,
                                         max_tokens=180)
        conv_manager.messages = (
            [{"role": "system", "content": f"SUMMARY (auto): {summary}"}] +
            conv_manager.messages[-6:]
        )
        print(f"---Periodic summary applied after run {conv_manager.run_count}---")
        print(summary)

    return assistant_text


In [23]:
cm = ConversationManager()
samples = [
    "Hi, I'm Kshitij. I need an SDE resume review and help with projects.",
    "I have a ML project that detects defects in train wagons. How do I process video frames?",
    "Can you summarize my chat and extract my contact info? My email is ksh@example.com, phone 9876543210, I live in Delhi and I'm 23."
]

# Simulate runs with periodic summarization every 3 turns
for i, s in enumerate(samples, 1):
    print(f"\n=== Run {i} ===")
    out = run_with_periodic_summarization(client, cm, s, k=3)
    print("Assistant:", out[:200], "...\n")   # print first 200 chars

# Show truncation options
print("\n---Keep last 2 turns---")
cm.keep_last_n(4)  # 2 turns = 4 messages (user+assistant pairs)
for m in cm.messages:
    print(m)

print("\n---Truncate by chars (keep last ~200 chars)---")
cm.truncate_by_chars(200)
for m in cm.messages:
    print(m)



=== Run 1 ===
Assistant: Hello Kshitij, nice to meet you. I'd be happy to help you with your SDE resume review and suggest some projects to improve your skills.

To get started, can you please share your current resume with m ...


=== Run 2 ===
Assistant: That sounds like an interesting project. Processing video frames for defect detection in train wagons involves several steps. Here's a high-level overview of the process:

1. **Video Capture and Prepr ...


=== Run 3 ===
---Periodic summary applied after run 3---
**Summary:**

Kshitij, a 23-year-old from Delhi, sought help with an SDE resume review and project suggestions. He shared a machine learning project that detects defects in train wagons and asked for assistance with processing video frames. I provided a high-level overview of the steps involved in this process.
Assistant: I can summarize our chat and provide some general information, but I don't have the ability to extract or store your contact information. I'm a large lang

In [24]:
import json

schema = {
    "name": "extract_user_info",
    "description": "Extract user details from conversation",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string", "description": "Full name of the user"},
            "email": {"type": "string", "description": "Email address of the user"},
            "phone": {"type": "string", "description": "Phone number of the user"},
            "location": {"type": "string", "description": "Location (city or country) of the user"},
            "age": {"type": "integer", "description": "Age of the user in years"}
        },
        "required": ["name", "email", "phone", "location", "age"]
    }
}


In [28]:
def extract_info(client, text, model="llama-3.1-8b-instant"):
    """
    Extract structured user info (name, email, phone, location, age) as JSON.
    """
    messages = [
        {"role": "system", "content": "Extract user details and return ONLY valid JSON strictly following the schema."},
        {"role": "user", "content": text}
    ]

    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        response_format={"type": "json_object"}   # ✅ Force JSON output
    )

    json_output = resp.choices[0].message.content
    try:
        return json.loads(json_output)
    except Exception as e:
        print("Parsing error:", e)
        return {}


In [29]:
samples = [
    "Hi, I’m Kshitij Sharma, 23 years old from Delhi. My email is ksh@example.com and my phone is 9876543210.",
    "Hey, my name is Priya Mehta. I live in Mumbai, I’m 29, you can reach me at priya.mehta@mail.com or call me on 9123456789.",
    "This is John Doe, age 35, based in Bangalore. Contact me at john@doe.com, phone 9988776655."
]

for s in samples:
    print("\nChat:", s)
    extracted = extract_info(client, s)
    print("Extracted:", extracted)



Chat: Hi, I’m Kshitij Sharma, 23 years old from Delhi. My email is ksh@example.com and my phone is 9876543210.
Extracted: {'username': 'Kshitij Sharma', 'age': 23, 'location': 'Delhi', 'email': 'ksh@example.com', 'phone': '9876543210'}

Chat: Hey, my name is Priya Mehta. I live in Mumbai, I’m 29, you can reach me at priya.mehta@mail.com or call me on 9123456789.
Extracted: {'name': 'Priya Mehta', 'age': 29, 'location': 'Mumbai', 'contact': {'email': 'priya.mehta@mail.com', 'phone': '9123456789'}}

Chat: This is John Doe, age 35, based in Bangalore. Contact me at john@doe.com, phone 9988776655.
Extracted: {'name': 'John Doe', 'age': 35, 'location': 'Bangalore', 'email': 'john@doe.com', 'phone': '9988776655'}


In [30]:
from jsonschema import validate

for s in samples:
    extracted = extract_info(client, s)
    try:
        validate(instance=extracted, schema=schema["parameters"])
        print("Valid ✅", extracted)
    except Exception as e:
        print("Invalid ❌", e)


Invalid ❌ 'location' is a required property

Failed validating 'required' in schema:
    {'type': 'object',
     'properties': {'name': {'type': 'string',
                             'description': 'Full name of the user'},
                    'email': {'type': 'string',
                              'description': 'Email address of the user'},
                    'phone': {'type': 'string',
                              'description': 'Phone number of the user'},
                    'location': {'type': 'string',
                                 'description': 'Location (city or '
                                                'country) of the user'},
                    'age': {'type': 'integer',
                            'description': 'Age of the user in years'}},
     'required': ['name', 'email', 'phone', 'location', 'age']}

On instance:
    {'name': 'Kshitij Sharma',
     'age': 23,
     'city': 'Delhi',
     'email': 'ksh@example.com',
     'phone': '9876543210'}
Invalid ❌